# Import Modules

In [2]:
import os
import sys
import os.path as path
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt
plt.style.use('seaborn-white')
from datetime import date, datetime, timedelta, time

# Paths, File Names

### DFM Files

In [3]:
dfm_files_loc = '/p/lustre1/mirocha2/SJSU_DATA/akochanski/PGnE_climo/dfm'

#### Global File Names

In [4]:
file_start_time = '1989-09-01_00:00:00'
file_end_time   = '1990-12-30_00:00:00'

In [5]:
start_time = datetime.fromisoformat(file_start_time)
end_time = datetime.fromisoformat(file_end_time)
time_interval = timedelta (days=5)
#start_time, end_time, time_interval

In [6]:
current_time = start_time
file_time_list = [current_time.isoformat(sep = '_')]
while (current_time < end_time):
    current_time += time_interval
    #print ('current_time -> {}'.format(current_time.isoformat(sep = '_')))
    file_time_list.append(current_time.isoformat(sep = '_'))

In [7]:
dfm_file_names = []
for file_time in file_time_list:
    dfm_file_name = 'wrfout_d03_' + file_time + '_dfm.nc'
    dfm_file_names.append(dfm_file_name)

In [8]:
#dfm_file_names
#len(dfm_file_names)

#### Check if the filenames exist

In [9]:
'''
for file_count, dfm_file_name in enumerate(dfm_file_names):
    print('file_count', file_count)
    if (path.exists(path.join(dfm_files_loc, dfm_file_name))):
        print('File : {} exists at the location : {}'.format(
            dfm_file_name, dfm_files_loc))
    else:
        print('File : {} not found at the location : {}'.format(
            dfm_file_name, dfm_files_loc))
'''

"\nfor file_count, dfm_file_name in enumerate(dfm_file_names):\n    print('file_count', file_count)\n    if (path.exists(path.join(dfm_files_loc, dfm_file_name))):\n        print('File : {} exists at the location : {}'.format(\n            dfm_file_name, dfm_files_loc))\n    else:\n        print('File : {} not found at the location : {}'.format(\n            dfm_file_name, dfm_files_loc))\n"

# Read Data From A Single DFM File

In [10]:
file_to_read = dfm_file_names[0]
file_time = file_to_read[11:-7]
ref_time_for_FM = '1989-09-05_23:00:00'
time_unix   = '1970-01-01_00:00:00'

#### Define time stamps as hours since unix time

In [11]:
file_time_stamp = datetime.fromisoformat(file_time) - datetime.fromisoformat(time_unix)
file_time_stamp = int(file_time_stamp.total_seconds()/3600)

In [12]:
FM_time_stamp = datetime.fromisoformat(ref_time_for_FM) - datetime.fromisoformat(time_unix)
FM_time_stamp = int(FM_time_stamp.total_seconds()/3600)

In [13]:
print('file_time_stamp : %d, FM_time_stamp : %d'%(file_time_stamp, FM_time_stamp))

file_time_stamp : 172392, FM_time_stamp : 172511


#### Define indixes for extracting data

In [14]:
FM_time_index = FM_time_stamp - file_time_stamp
FM_time_index

119

In [15]:
n_history = 20
interval_history = -3
AtmData_indices = np.arange(FM_time_index, FM_time_index-n_history, interval_history)
AtmData_indices = list(np.sort(AtmData_indices)[:-1])

In [16]:
AtmData_indices

[101, 104, 107, 110, 113, 116]

#### Data fields of interest

In [17]:
FM_field = 'mean_wtd_moisture_10hr'
steady_fields = ['latitude', 'longitude']
unsteady_fields = ['eastward_10m_wind', 'northward_10m_wind',\
                   'air_temperature_2m','accumulated_precipitation_amount'] 

#### Read The Data

In [18]:
df = pd.DataFrame()

In [19]:
dfm_file_data = xr.open_dataset(path.join(dfm_files_loc, file_to_read))

In [20]:
df['lat'] = np.array(dfm_file_data['latitude']).flatten()
df['lon'] = np.array(dfm_file_data['longitude']).flatten()

In [21]:
FM_data = dfm_file_data['mean_wtd_moisture_10hr'].isel(time=FM_time_index)
FM_Data = np.array(FM_data).flatten()
df['FM_10hr_' + str(FM_time_index - FM_time_index)] = FM_Data

In [22]:
for hist_data_ind in AtmData_indices:
    #hist_data_ind = AtmData_indices[0]
    hist_time_stamp = file_time_stamp + hist_data_ind

    U10_data = np.array(dfm_file_data['eastward_10m_wind'].isel(
                        time=hist_data_ind)).flatten()
    V10_data = np.array(dfm_file_data['northward_10m_wind'].isel(
                        time=hist_data_ind)).flatten()
    T2_data  = np.array(dfm_file_data['air_temperature_2m'].isel(
                        time=hist_data_ind)).flatten()
    Precip_data = np.array(dfm_file_data['accumulated_precipitation_amount'].isel(
                        time=hist_data_ind)).flatten()

    df['U10[%s]'%str(hist_data_ind - FM_time_index)] = U10_data
    df['V10[%s]'%str(hist_data_ind - FM_time_index)] = V10_data
    df['T2[%s]'%str(hist_data_ind - FM_time_index)] = T2_data
    df['Precip[%s]'%str(hist_data_ind - FM_time_index)] = Precip_data

In [23]:
df[40590:40600]

,lat,lon,FM_10hr_0,U10[-18],V10[-18],T2[-18],Precip[-18],U10[-15],V10[-15],T2[-15],...,T2[-9],Precip[-9],U10[-6],V10[-6],T2[-6],Precip[-6],U10[-3],V10[-3],T2[-3],Precip[-3]
40590,35.517555,-121.035492,0.104995,1.001995,-0.635718,13.674957,0.000151,0.721998,-0.724310,12.878754,...,11.705688,0.000151,1.801164,-0.808555,17.952911,0.000151,2.752246,-1.059031,21.081543,0.000151
40591,35.517536,-121.013245,0.103625,0.812303,-0.242862,13.846771,0.000050,0.869555,-0.550958,13.015991,...,11.839020,0.000050,1.525809,-0.720257,19.488708,0.000050,2.499162,-0.605951,22.598511,0.000050
40592,35.517529,-120.990997,0.098448,0.485963,-0.037503,14.273193,0.000028,0.746158,-0.611051,13.404388,...,11.470459,0.000028,1.509356,-0.779113,20.578064,0.000028,2.457479,-0.453497,24.702057,0.000028
40593,35.517506,-120.968735,0.094208,-0.381711,0.725962,14.707062,0.000048,0.344997,-0.327270,13.838104,...,12.350677,0.000048,1.407859,-0.238341,21.469452,0.000048,2.198458,0.018919,25.995483,0.000048
40594,35.517479,-120.946487,0.088373,-1.043693,1.097154,14.729462,0.000000,-0.141151,-0.221582,13.667816,...,13.017395,0.000000,1.242805,0.269898,22.008698,0.000000,1.826316,0.663107,26.562775,0.000000
40595,35.517452,-120.924225,0.085051,-1.207483,1.010152,14.369080,0.000000,-0.453239,-0.109485,13.289062,...,15.310852,0.000000,1.105457,0.581854,22.450470,0.000000,1.551614,1.191232,26.482758,0.000000
40596,35.517414,-120.901978,0.078415,-1.032169,0.514973,14.412354,0.000000,-0.401610,0.119516,13.953461,...,19.366669,0.000000,1.257412,0.648969,22.881500,0.000000,1.719669,1.404052,26.135071,0.000000
40597,35.517380,-120.879730,0.067936,-0.692086,-0.125240,16.130707,0.000000,-0.662902,-1.216859,19.108734,...,19.500946,0.000000,1.226674,0.704142,24.172882,0.000000,2.228383,1.421653,26.023560,0.000000
40598,35.517338,-120.857483,0.064074,0.312117,-0.672418,17.108307,0.000000,-1.602128,-2.752957,18.970154,...,17.330261,0.000000,-0.569629,-0.384460,27.469818,0.000000,2.691557,1.325173,26.346252,0.000000
40599,35.517288,-120.835236,0.061938,0.988312,-0.422380,17.263763,0.000000,-2.072484,-2.290591,17.267181,...,16.358856,0.000000,-1.565127,-1.320302,27.128632,0.000000,2.672156,1.620060,27.828430,0.000000
